In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store, fetch_predictions
import pandas as pd
from datetime import datetime, timedelta, timezone


def fetch_days_data(days):
    current_date = pd.to_datetime(datetime.now(timezone.utc))
    fetch_data_from = current_date - timedelta(days=(365+days)) 
    fetch_data_to = current_date - timedelta(days=365)
    print(fetch_data_from, fetch_data_to)
    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_NAME,
        version=1
    )

    query = fg.select_all()
    # query = query.filter((fg.pickup_hour >= fetch_data_from))
    df = query.read()
    cond = (df["pickup_hour"] >= fetch_data_from) & (df["pickup_hour"] <= fetch_data_to)
    return df[cond]

In [4]:
ts_data = fetch_days_data(180)

2023-09-05 07:18:49.792407+00:00 2024-03-03 07:18:49.792407+00:00
2025-03-03 02:18:49,793 INFO: Initializing external client
2025-03-03 02:18:49,793 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 02:18:50,680 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215651
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.99s) from Hopsworks, using Hopsworks Feature Query Service.   Reading data from Hopsworks, using Hopsworks Feature Query Service...   Reading data from Hopsworks, using Hopsworks Feature Query Service.   Reading data from Hopsworks, using Hopsworks Feature Query Service...   


In [5]:
from src.data_utils import transform_ts_data_info_features_and_target

features, targets = transform_ts_data_info_features_and_target(ts_data, window_size=24*28, step_size=23)

In [6]:
from src.pipeline_utils import get_pipeline
pipeline = get_pipeline()
pipeline.fit(features, targets)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.048819 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 171646
[LightGBM] [Info] Number of data points in the train set: 24440, number of used features: 675
[LightGBM] [Info] Start training from score 16.812398


Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x1323fb700>)),
                ('temporalfeatureengineer', TemporalFeatureEngineer()),
                ('lgbmregressor', LGBMRegressor())])

In [7]:
from sklearn.metrics import mean_absolute_error
predictions = pipeline.predict(features)

In [8]:
test_mae = mean_absolute_error(targets, predictions)
print(f"{test_mae:.4f}")

2.9751


In [9]:
from src.inference import load_metrics_from_registry 

metric = load_metrics_from_registry()

2025-03-03 02:19:18,772 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 02:19:18,775 INFO: Initializing external client
2025-03-03 02:19:18,775 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 02:19:19,580 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215651


In [10]:
metric

{'test_mae': 1.935489834251996}

In [11]:
from src.inference import load_model_from_registry
model = load_model_from_registry()

2025-03-03 02:19:24,540 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 02:19:24,543 INFO: Initializing external client
2025-03-03 02:19:24,543 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 02:19:25,297 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215651


In [12]:
import joblib  
import src.config
# Save the pipeline  
joblib.dump(pipeline, config.MODELS_DIR / "lgb_model.pkl")

['/Users/harshithharsha/MY DOCS/APL-AT-S/sp25_taxi-main/models/lgb_model.pkl']

In [13]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(features)
output_schema = Schema(targets)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

In [14]:
model_registry = project.get_model_registry()
model_registry

NameError: name 'project' is not defined

In [15]:
modelv2 = model_registry.sklearn.create_model(
    name="taxi_demand_predictor_next_hour",
    metrics={"test_mae": test_mae},
    description="LightGBM regressor V2",
    input_example=features.sample(),
    model_schema=model_schema,
)

NameError: name 'model_registry' is not defined

In [16]:
modelv2.save('C:\\Users\\mkhaw\\github\\sp25_taxi\\models\\lgb_model.pkl')

NameError: name 'modelv2' is not defined

In [17]:
from src.inference import load_model_from_registry
model = load_model_from_registry(-1)

2025-03-03 02:19:46,031 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 02:19:46,034 INFO: Initializing external client
2025-03-03 02:19:46,034 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 02:19:46,824 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215651


In [18]:
models = model_registry.get_models(name=config.MODEL_NAME)

NameError: name 'model_registry' is not defined

In [19]:
models

NameError: name 'models' is not defined

In [20]:
max(models, key=lambda model: model.version)


NameError: name 'models' is not defined

In [21]:
load_metrics_from_registry()

2025-03-03 02:19:56,637 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 02:19:56,639 INFO: Initializing external client
2025-03-03 02:19:56,639 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 02:19:57,418 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215651


{'test_mae': 1.935489834251996}